In [2]:
#Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests

In [3]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
regions_df = pd.read_csv('Resources/noc_regions.csv')
host_cities_df = pd.read_csv('Resources/olym.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
#Jumaan start here (Question 1)

In [ ]:
#Jumaan end here

In [ ]:
#Timmy start here (Question 2)

In [ ]:
#Timmy end here

In [ ]:
#Raj start here (Question 3)

In [ ]:
#Raj end here

In [ ]:
#Fatma start here (Question 4)

In [4]:
olympics_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [6]:
host_cities_df.head()
host_cities_df = host_cities_df[["City", "Country"    ]]
host_cities_df.head()

,City,Country
0,Athens,Greece
1,Paris,France
2,St. Louis,United States
3,London,United Kingdom
4,Stockholm,Sweden


In [7]:
olympics_country_df = pd.merge(olympics_df, host_cities_df, how="left", on=["City"])
olympics_country_df=olympics_country_df.drop_duplicates()

olympics_country_df.head()
# remove duplicates

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,Spain
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,United Kingdom
4,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,NaN
5,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,France
7,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Canada


In [8]:
olympics_country_df.groupby(["Country", "Year","Season", "Team"]).count()

ID  Name  Sex  Age  Height  Weight  \
Country    Year Season Team                                                  
Australia  1956 Summer Afghanistan      12    12   12    0       0       0   
                       Aretusa           3     3    3    3       2       2   
                       Argentina        26    26   26   22       9      10   
                       Australia       395   395  395  322     121     119   
                       Austria          49    49   49   48      19      18   
...                                    ...   ...  ...  ...     ...     ...   
Yugoslavia 1984 Winter West Germany-1    8     8    8    8       8       8   
                       West Germany-2    8     8    8    8       8       8   
                       Yugoslavia      103   103  103  103      96      96   
                       Yugoslavia-1      6     6    6    6       5       5   
                       Yugoslavia-2      6     6    6    6       4       4   

                                       NOC  Games  City  Sport  Event  Medal  
Country    Year Season Team                                                   
Australia  1956 Summer Afghanistan      12     12    12     12     12      0  
                       Aretusa           3      3     3      3      3      0  
                       Argentina        26     26    26     26     26      2  
                       Australia       395    395   395    395    395     62  
                       Austria          49     49    49     49     49      4  
...                                    ...    ...   ...    ...    ...    ...  
Yugoslavia 1984 Winter West Germany-1    8      8     8      8      8      2  
                       West Germany-2    8      8     8      8      8      0  
                       Yugoslavia      103    103   103    103    103      1  
                       Yugoslavia-1      6      6     6      6      6      0  
                       Yugoslavia-2      6      6     6      6      6      0  

[4900 rows x 12 columns]

In [ ]:
#Fatma end here